In [7]:
import os
import shutil

In [8]:
%store -r 
#restore train, validate, and test dataframes from other notebook
train.head()

,file_name,hair_color,eyeglasses,smiling,young,human
2308,2531,2,1,1,1,-1
82,92,1,1,1,1,-1
4401,4830,-1,-1,-1,1,1
817,907,0,-1,-1,1,1
471,522,1,-1,1,1,1


In [6]:
def reset_images_to_main_dir(delete_dirs=True):
    dest_dir = './images/'
    walker = os.walk(dest_dir)
    rem_dirs = walker.__next__()[1]
 
    for data in walker:
        for files in data[2]:
            try:
                shutil.move(data[0] + os.sep + files, dest_dir)
            except shutil.Error:
                print(shutil.Error)
                continue
    if delete_dirs:
        for dirs in rem_dirs:
            shutil.rmtree(dest_dir + os.sep + dirs)

def symlink_classes_images(train, validate, test, reset_images=False, 
                           delete_dirs=False):
    if reset_images:
        reset_images_to_main_dir(delete_dirs=delete_dirs)
    imagesPath = os.path.abspath('./images/')
    trainPath = os.path.abspath('./images/train/eyeglasses/')
    validatePath = os.path.abspath('./images/validate/eyeglasses/')
    testPath = os.path.abspath('./images/test/eyeglasses/')
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            if int(row['eyeglasses'])==-1:
                pathAppend = 'no_glasses/'
            elif int(row['eyeglasses']==1):
                pathAppend = 'glasses/'
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainClassPath = os.path.join(trainPath, pathAppend)
            trainFilePath = os.path.join(trainClassPath, fileName)
            validateClassPath = os.path.join(validatePath, pathAppend)
            validateFilePath = os.path.join(validateClassPath, fileName)
            testClassPath = os.path.join(testPath, pathAppend)
            testFilePath = os.path.join(testClassPath, fileName)
            classPaths = [trainClassPath, validateClassPath, testClassPath]
            for path in classPaths:
                if os.path.isdir(path)==False:
                    os.mkdir(path)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                os.symlink(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                os.symlink(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                os.symlink(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                os.symlink(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)
                
symlink_classes_images(train, validate, test)

In [9]:
train_glasses = train[['file_name', 'eyeglasses']]
train_glasses = train_glasses.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)
validate_glasses = validate[['file_name', 'eyeglasses']]
validate_glasses = validate_glasses.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)
test_glasses = test[['file_name', 'eyeglasses']]
test_glasses = test_glasses.replace(to_replace=1, value=1).replace(to_replace=-1, value=0).reset_index(drop=True)

In [10]:
n_glasses = train_glasses['eyeglasses'].value_counts()
print(n_glasses)

0    1947
1     792
Name: eyeglasses, dtype: int64


In [11]:
#https://jovianlin.io/cat-crossentropy-vs-sparse-cat-crossentropy/

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

K.set_image_dim_ordering('th')

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(3, 128, 128)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


In [12]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def get_ImageDataGenerator(shear_range=0, 
                           zoom_range=0, 
                           horizontal_flip=False):
    datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0,
                zoom_range=0,
                horizontal_flip=False,
                data_format='channels_first')
    
    return datagen

def get_flow_from_directory(ImageDataGenerator,
                            img_dir,
                            target_size=(128,128),
                            class_mode='binary',
                            batch_size=32,
                            shuffle=True,
                            seed=123):
    gen = ImageDataGenerator.flow_from_directory(
                            directory=img_dir,
                            target_size=target_size,
                            class_mode=class_mode,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            seed=seed,
                            follow_links=True)
    return gen


# this is the augmentation configuration we will use for training
train_datagen = get_ImageDataGenerator(shear_range=0,
                                       zoom_range=0,
                                       horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = get_ImageDataGenerator()
#Takes the dataframe and the path to a directory and 
#generates batches of augmented/normalized data.

trainPath = './images/train/eyeglasses/'
validatePath = './images/validate/eyeglasses/'
batch_size = 32
target_size=(128,128)
class_mode='binary'
shuffle=True
seed=123

train_gen = get_flow_from_directory(train_datagen,
                                    trainPath,
                                    target_size,
                                    class_mode,
                                    batch_size,
                                    shuffle,
                                    seed)

validation_gen = get_flow_from_directory(test_datagen,
                                    validatePath,
                                    target_size,
                                    class_mode,
                                    batch_size,
                                    shuffle,
                                    seed)

Found 2739 images belonging to 2 classes.
Found 913 images belonging to 2 classes.


In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight

earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=0,
                              verbose=1, 
                              mode='auto',
                              restore_best_weights=True)

LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=0, 
                       verbose=1, 
                       mode='auto', 
                       min_delta=0.01, 
                       cooldown=0, 
                       min_lr=0.001)
train_gen.reset()
validation_gen.reset()

train_steps=train_gen.n/train_gen.batch_size
valid_steps=validation_gen.n/validation_gen.batch_size

In [ ]:
print(train_steps, valid_steps)
print(train_steps*train_gen.batch_size, valid_steps*validation_gen.batch_size)
model.summary()

In [ ]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=validation_gen,
                    validation_steps=valid_steps,
                    epochs=50,
                    callbacks=[earlyStopping, LR],
                    class_weight = 'auto')

#model.save_weights('.h5')
model.save_weights('glasses_detection_from_dir.h5')

In [ ]:
validation_gen = get_flow_from_directory(test_datagen,
                                        validatePath,
                                        target_size,
                                        class_mode,
                                        1,
                                        False)

validation_gen.reset()

model.evaluate_generator(generator=validation_gen, steps=913, verbose=1)

In [ ]:
testPath = './images/test/'


test_gen = get_flow_from_directory(test_datagen,
                                    testPath,
                                    target_size,
                                    class_mode,
                                    1,
                                    False)

test_gen.reset()

model.evaluate_generator(generator=test_gen, steps=913, verbose=1)

In [ ]:
test_gen.reset()
probabilities = model.predict_generator(test_gen,verbose=1,steps=len(test_gen))

In [ ]:
import pandas as pd
import numpy as np
output_df = pd.DataFrame(
    {'filename': test_gen.filenames,
     'y_true': test_gen.classes,
     'y_pred': list(map(int, np.rint(probabilities).flatten()))
    })

In [ ]:
output_df_wrong = output_df.query('y_true != y_pred')
output_df_wrong

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = test_gen.classes
y_pred = np.rint(probabilities)
confusion_matrix(y_true=y_true, y_pred=y_pred)